# PFF Data (2006-2023)
Now, we can add player and team-related grades acquired from [Pro Football Focus](https://www.pff.com/).

In [47]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# display
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

# global random_state
random_state = 9

In [48]:
# load in master data
master = pd.read_csv('./data/final_data/master.csv')

# drop cols
# scoring = ['standard', 'half-ppr', '6']
# for col in master.columns:
#     if any([x in col for x in scoring]):
#         master.drop(col, axis=1, inplace=True)

# create positional subsets
qb = master[master['Pos'] == 'QB']
rb = master[master['Pos'] == 'RB']
wr = master[master['Pos'] == 'WR']
te = master[master['Pos'] == 'TE']

In [49]:
qb.shape, rb.shape, wr.shape, te.shape

((4148, 132), (9297, 132), (9827, 132), (5468, 132))

## Passing Data

In [50]:
# drop 'Rec' cols
qb = qb.drop([col for col in qb.columns if 'Rec' in col], axis=1)

In [51]:
# get all passing data files
file_paths = [os.path.join('./data/pff_data/passing_data', file) for file in os.listdir('./data/pff_data/passing_data') if file.endswith('.csv')]

# list to hold dfs
dfs = []

# read each file into a dataframe
for file_path in file_paths:
    # get year as string from filename
    year = file_path[-8:-4]
    
    # load each season into a df
    data = pd.read_csv(file_path)
    
    # add year column
    data['Year'] = int(year)
    
    # add df to list
    dfs.append(data)

# stack dataframes together
pass_df = pd.concat(dfs, axis=0, ignore_index=True)

# add cols
pass_df['Dropback%'] = pass_df['dropbacks'] / pass_df['passing_snaps']
pass_df['Aimed_passes%'] = pass_df['aimed_passes'] / pass_df['attempts']
pass_df['Dropped_passes%'] = pass_df['drops'] / pass_df['aimed_passes']
pass_df['Batted_passes%'] = pass_df['bats'] / pass_df['aimed_passes']
pass_df['Thrown_away%'] = pass_df['thrown_aways'] / pass_df['passing_snaps']
pass_df['Pressure%'] = pass_df['def_gen_pressures'] / pass_df['passing_snaps']
pass_df['Scramble%'] = pass_df['scrambles'] / pass_df['passing_snaps']
pass_df['Sack%'] = pass_df['sacks'] / pass_df['passing_snaps']
pass_df['Pressure_to_sack%'] = pass_df['sacks'] / pass_df['def_gen_pressures']
pass_df['BTT%'] = pass_df['big_time_throws'] / pass_df['aimed_passes']
pass_df['TWP%'] = pass_df['turnover_worthy_plays'] / pass_df['aimed_passes']
pass_df['First_down%'] = pass_df['first_downs'] / pass_df['attempts']

# # drop columns
pass_df = pass_df.drop(columns=['player_id', 'position', 'team_name', 'player_game_count', 'aimed_passes', 'attempts', 'bats', 'big_time_throws', 'btt_rate', 'completion_percent', 'completions', 'declined_penalties', 'def_gen_pressures', 'drop_rate', 'drops', 'grades_offense', 'grades_run', 
                      'grades_hands_fumble', 'franchise_id', 'hit_as_threw', 'interceptions', 'penalties', 'pressure_to_sack_rate', 'qb_rating', 'sack_percent', 'sacks', 'scrambles', 'spikes', 'thrown_aways', 'touchdowns', 'turnover_worthy_plays', 'twp_rate', 'yards', 'ypa', 'first_downs'])

# add 'Pass' to the beginning of each column name
pass_df.columns = ['Pass_' + col for col in pass_df.columns]

# remove 'Pass' from 'Player' and 'Year'
pass_df['Player'] = pass_df['Pass_player']
pass_df['Year'] = pass_df['Pass_Year']
pass_df = pass_df.drop(columns=['Pass_player', 'Pass_Year'])

# check
pass_df.head()

,Pass_accuracy_percent,Pass_avg_depth_of_target,Pass_avg_time_to_throw,Pass_dropbacks,Pass_grades_pass,Pass_passing_snaps,Pass_Dropback%,Pass_Aimed_passes%,Pass_Dropped_passes%,Pass_Batted_passes%,Pass_Thrown_away%,Pass_Pressure%,Pass_Scramble%,Pass_Sack%,Pass_Pressure_to_sack%,Pass_BTT%,Pass_TWP%,Pass_First_down%,Player,Year
0,66.2,9.0,2.52,638,57.4,665,0.959398,0.951060,0.073756,0.020583,0.018045,0.243609,0.006015,0.031579,0.129630,0.037736,0.046312,0.306688,Brett Favre,2006
1,71.1,9.3,2.79,677,65.5,714,0.948179,0.939597,0.046429,0.016071,0.026611,0.343137,0.025210,0.088235,0.257143,0.042857,0.058929,0.370805,Jon Kitna,2006
2,70.5,9.5,2.51,642,71.5,675,0.951111,0.950680,0.042934,0.008945,0.020741,0.257778,0.005926,0.074074,0.287356,0.046512,0.035778,0.365646,Marc Bulger,2006
3,74.6,10.4,2.60,577,93.4,602,0.958472,0.946140,0.058824,0.013283,0.029900,0.252492,0.009967,0.023256,0.092105,0.077799,0.026565,0.439856,Peyton Manning,2006
4,78.5,7.9,2.58,584,88.6,606,0.963696,0.913357,0.081028,0.045455,0.031353,0.262376,0.018152,0.029703,0.113208,0.061265,0.035573,0.366426,Drew Brees,2006


In [52]:
# merge with qb
qb = qb.merge(pass_df, on=['Player', 'Year'], how='left')

# check shape
qb.shape

(4148, 141)

In [53]:
qb.isna().sum().to_frame().T

,Player,Tm,Pos,Age,G,GS,Pass_Cmp,Pass_Att,Pass_Yds,Pass_TD,Pass_Int,Rush_Att,Rush_Yds,Rush_Y/A,Rush_TD,Fmb,FmbLost,Key,Year,Scrim_Yds,Scrim_TD,num_games,games_played_pct,games_started_pct,ProBowl,AllPro,Exp,New_Team,Will_be_on_New_Team,Traded,Pass_Y/A,Touches,Cmp%,Catch%,Pass_Cmp_per_game,Pass_Att_per_game,Pass_Yds_per_game,Pass_TD_per_game,Pass_Int_per_game,Rush_Att_per_game,Rush_Yds_per_game,Rush_TD_per_game,Fmb_per_game,FmbLost_per_game,Scrim_Yds_per_game,Scrim_TD_per_game,Touches_per_game,Points_standard,Points_half-ppr,Points_ppr,Points_6,PPG_standard,PPG_half-ppr,PPG_ppr,PPG_6,PPT_standard,PPT_half-ppr,PPT_ppr,PPT_6,SeasonOvrRank_standard,SeasonOvrRank_half-ppr,SeasonOvrRank_ppr,SeasonOvrRank_6,SeasonPosRank_standard,SeasonPosRank_half-ppr,SeasonPosRank_ppr,SeasonPosRank_6,PPGOvrRank_standard,PPGOvrRank_half-ppr,PPGOvrRank_ppr,PPGOvrRank_6,PPGPosRank_standard,PPGPosRank_half-ppr,PPGPosRank_ppr,PPGPosRank_6,PPTOvrRank_standard,PPTOvrRank_half-ppr,PPTOvrRank_ppr,PPTOvrRank_6,PPTPosRank_standard,PPTPosRank_half-ppr,PPTPosRank_ppr,PPTPosRank_6,Points_VORP_standard_10tm,Points_VORP_half-ppr_10tm,Points_VORP_ppr_10tm,Points_VORP_6_10tm,PPG_VORP_standard_10tm,PPG_VORP_half-ppr_10tm,PPG_VORP_ppr_10tm,PPG_VORP_6_10tm,Points_VORP_standard_12tm,Points_VORP_half-ppr_12tm,Points_VORP_ppr_12tm,Points_VORP_6_12tm,PPG_VORP_standard_12tm,PPG_VORP_half-ppr_12tm,PPG_VORP_ppr_12tm,PPG_VORP_6_12tm,Points_VORP_standard_10tm_3WR,Points_VORP_half-ppr_10tm_3WR,Points_VORP_ppr_10tm_3WR,Points_VORP_6_10tm_3WR,PPG_VORP_standard_10tm_3WR,PPG_VORP_half-ppr_10tm_3WR,PPG_VORP_ppr_10tm_3WR,PPG_VORP_6_10tm_3WR,Points_VORP_standard_12tm_3WR,Points_VORP_half-ppr_12tm_3WR,Points_VORP_ppr_12tm_3WR,Points_VORP_6_12tm_3WR,PPG_VORP_standard_12tm_3WR,PPG_VORP_half-ppr_12tm_3WR,PPG_VORP_ppr_12tm_3WR,PPG_VORP_6_12tm_3WR,SeasonTarget_standard,SeasonTarget_half-ppr,SeasonTarget_ppr,SeasonTarget_6,PPGTarget_standard,PPGTarget_half-ppr,PPGTarget_ppr,PPGTarget_6,Pass_accuracy_percent,Pass_avg_depth_of_target,Pass_avg_time_to_throw,Pass_dropbacks,Pass_grades_pass,Pass_passing_snaps,Pass_Dropback%,Pass_Aimed_passes%,Pass_Dropped_passes%,Pass_Batted_passes%,Pass_Thrown_away%,Pass_Pressure%,Pass_Scramble%,Pass_Sack%,Pass_Pressure_to_sack%,Pass_BTT%,Pass_TWP%,Pass_First_down%
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,9,9,9,9,9,9,9,9,9,9,9,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,758,758,758,758,758,758,758,758,2854,2854,2852,2852,2852,2852,2852,2852,2854,2854,2852,2852,2852,2852,2890,2854,2854,2852


## Rushing Data
Now, we will add the advanced rushing metrics to both the QB and RB dataframes.

In [54]:
# drop 'Pass' cols
rb = rb.drop([col for col in rb.columns if 'Pass' in col], axis=1)
wr = wr.drop([col for col in wr.columns if 'Pass' in col], axis=1)
te = te.drop([col for col in te.columns if 'Pass' in col], axis=1)

In [55]:
# get all rushing data files
file_paths = [os.path.join('./data/pff_data/rushing_data', file) for file in os.listdir('./data/pff_data/rushing_data') if file.endswith('.csv')]

# list to hold dfs
dfs = []

# read each file into a dataframe
for file_path in file_paths:
    # get year as string from filename
    year = file_path[-8:-4]
    
    # load each season into a df
    data = pd.read_csv(file_path)
    
    # add year column
    data['Year'] = int(year)
    
    # add df to list
    dfs.append(data)

# stack dataframes together
rush_df = pd.concat(dfs, axis=0, ignore_index=True)

# add cols
rush_df['Team_Rush%'] = rush_df['attempts'] / rush_df['run_plays']
rush_df['Avoided_tackles_per_attempt'] = rush_df['avoided_tackles'] / rush_df['attempts']
rush_df['10+_yard_run%'] = rush_df['explosive'] / rush_df['attempts']
rush_df['15+_yard_run%'] = rush_df['breakaway_attempts'] / rush_df['attempts']
rush_df['15+_yard_run_yards%'] = rush_df['breakaway_yards'] / rush_df['yards']
rush_df['First_down%'] = rush_df['first_downs'] / rush_df['attempts']
rush_df['Gap%'] = rush_df['gap_attempts'] / rush_df['attempts']
rush_df['Zone%'] = rush_df['zone_attempts'] / rush_df['attempts']
rush_df['YCO_per_attempt'] = rush_df['yards_after_contact'] / rush_df['attempts']

# drop columns
rush_df = rush_df.drop(columns=['player_id', 'position', 'team_name', 'player_game_count', 'attempts', 'avoided_tackles', 'breakaway_attempts', 'breakaway_percent', 'breakaway_yards', 'declined_penalties', 'designed_yards', 'drops', 'elu_recv_mtf', 'elu_rush_mtf', 'elu_yco', 
                                'explosive', 'first_downs', 'franchise_id', 'fumbles', 'gap_attempts', 'grades_offense_penalty', 'grades_pass', 'grades_pass_block', 'grades_pass_route', 'grades_run_block', 'penalties', 'rec_yards', 'receptions', 'routes', 'scramble_yards', 
                                'scrambles', 'targets', 'total_touches', 'touchdowns', 'yards', 'yards_after_contact', 'yco_attempt', 'ypa', 'yprr', 'run_plays', 'zone_attempts'])

# add 'Rush' to the beginning of each column name
rush_df.columns = ['Rush_' + col for col in rush_df.columns]

# remove 'Rush' from 'Player' and 'Year'
rush_df['Player'] = rush_df['Rush_player']
rush_df['Year'] = rush_df['Rush_Year']
rush_df = rush_df.drop(columns=['Rush_player', 'Rush_Year'])

# check
rush_df.head()

,Rush_elusive_rating,Rush_grades_hands_fumble,Rush_grades_offense,Rush_grades_run,Rush_longest,Rush_Team_Rush%,Rush_Avoided_tackles_per_attempt,Rush_10+_yard_run%,Rush_15+_yard_run%,Rush_15+_yard_run_yards%,Rush_First_down%,Rush_Gap%,Rush_Zone%,Rush_YCO_per_attempt,Player,Year
0,42.7,85.5,59.0,60.3,47,0.922395,0.146635,0.117788,0.067308,0.344885,0.216346,0.473558,0.420673,2.670673,Larry Johnson,2006
1,32.0,82.0,73.0,71.5,85,0.802784,0.106936,0.121387,0.066474,0.397018,0.231214,0.589595,0.297688,2.855491,LaDainian Tomlinson,2006
2,43.4,67.1,66.5,64.0,59,0.937500,0.165217,0.107246,0.046377,0.257853,0.211594,0.400000,0.460870,2.362319,Steven Jackson,2006
3,30.4,64.6,58.3,57.6,22,0.878866,0.126100,0.087977,0.026393,0.129106,0.196481,0.442815,0.489736,2.448680,Rudi Johnson,2006
4,22.7,79.4,49.6,46.8,18,0.925824,0.091988,0.059347,0.008902,0.044750,0.175074,0.326409,0.587537,2.421365,Edgerrin James,2006


In [56]:
# merge
qb = qb.merge(rush_df, on=['Player', 'Year'], how='left')
rb = rb.merge(rush_df, on=['Player', 'Year'], how='left')

# check shapes
qb.shape, rb.shape

((4148, 155), (9304, 135))

## Receiving Data

In [57]:
# get all receiving data files
file_paths = [os.path.join('./data/pff_data/receiving_data', file) for file in os.listdir('./data/pff_data/receiving_data') if file.endswith('.csv')]

# list to hold dfs
dfs = []

# read each file into a dataframe
for file_path in file_paths:
    # get year as string from filename
    year = file_path[-8:-4]
    
    # load each season into a df
    data = pd.read_csv(file_path)
    
    # add year column
    data['Year'] = int(year)
    
    # add df to list
    dfs.append(data)

# stack dataframes together
receiving_df = pd.concat(dfs, axis=0, ignore_index=True)

# add cols
receiving_df['Avoided_tackles_per_reception'] = receiving_df['avoided_tackles'] / receiving_df['receptions']
receiving_df['First_down%'] = receiving_df['first_downs'] / receiving_df['receptions']
receiving_df['Int_per_target'] = receiving_df['interceptions'] / receiving_df['targets']
receiving_df['YAC%'] = receiving_df['yards_after_catch'] / receiving_df['yards']

# drop columns
receiving_df = receiving_df.drop(columns=['player_id', 'position', 'team_name', 'player_game_count', 'avoided_tackles', 'contested_receptions', 'contested_targets', 'declined_penalties', 'drops', 'first_downs', 'franchise_id', 'fumbles', 'grades_pass_block', 
                                          'inline_snaps', 'pass_blocks', 'pass_plays', 'penalties', 'receptions', 'routes', 'slot_snaps', 'targets', 'touchdowns', 'wide_snaps', 'yards', 'yards_after_catch'])

# add 'Receiving' to the beginning of each column name
receiving_df.columns = ['Receiving_' + col for col in receiving_df.columns]

# remove 'Receiving' from 'Player' and 'Year'
receiving_df['Player'] = receiving_df['Receiving_player']
receiving_df['Year'] = receiving_df['Receiving_Year']
receiving_df = receiving_df.drop(columns=['Receiving_player', 'Receiving_Year'])

# check
receiving_df.head()

,Receiving_avg_depth_of_target,Receiving_caught_percent,Receiving_contested_catch_rate,Receiving_drop_rate,Receiving_grades_hands_drop,Receiving_grades_hands_fumble,Receiving_grades_offense,Receiving_grades_pass_route,Receiving_inline_rate,Receiving_interceptions,Receiving_longest,Receiving_pass_block_rate,Receiving_route_rate,Receiving_slot_rate,Receiving_targeted_qb_rating,Receiving_wide_rate,Receiving_yards_after_catch_per_reception,Receiving_yards_per_reception,Receiving_yprr,Receiving_Avoided_tackles_per_reception,Receiving_First_down%,Receiving_Int_per_target,Receiving_YAC%,Player,Year
0,14.5,53.1,NaN,6.9,69.6,29.9,80.7,80.5,0.0,4,67,0.0,95.1,13.7,84.9,86.3,3.2,13.2,1.93,0.042553,0.723404,0.022599,0.240097,Torry Holt,2006
1,12.0,55.8,NaN,7.1,65.3,58.8,80.7,81.0,0.2,5,82,0.0,95.7,62.9,84.8,36.8,5.7,14.1,2.10,0.141304,0.576087,0.030303,0.404009,Donald Driver,2006
2,10.2,65.6,NaN,8.0,58.6,84.0,79.9,79.6,0.6,9,53,0.0,95.4,28.3,74.1,70.8,4.3,11.2,2.24,0.135922,0.543689,0.057325,0.379221,Andre Johnson,2006
3,16.6,55.4,NaN,5.7,74.3,28.7,88.4,87.8,0.0,6,60,0.2,94.5,11.8,84.0,88.2,4.0,16.0,2.16,0.121951,0.841463,0.040541,0.247140,Roy Williams,2006
4,11.0,63.0,NaN,1.1,90.4,56.8,83.5,82.4,0.2,4,58,0.0,94.6,21.0,88.6,78.6,3.4,12.1,2.19,0.130435,0.565217,0.027397,0.279028,Laveranues Coles,2006


In [58]:
# merge
wr = wr.merge(receiving_df, on=['Player', 'Year'], how='left')
te = te.merge(receiving_df, on=['Player', 'Year'], how='left')

# merge with rb but drop duplicate cols
rb = rb.merge(receiving_df.drop(columns=['Receiving_grades_hands_fumble', 'Receiving_grades_offense']), on=['Player', 'Year'], how='left')

# check shapes
rb.shape, wr.shape, te.shape

((9321, 156), (9841, 144), (5478, 144))

## Blocking Data
This section calculates the normalized pass & run block grades for every team's __offensive line__.

In [13]:
# get all blocking data files
file_paths = [os.path.join('./data/pff_data/blocking_data', file) for file in os.listdir('./data/pff_data/blocking_data') if file.endswith('.csv')]

# list to hold dfs
dfs = []

# read each file into a dataframe
for file_path in file_paths:
    # get year as string from filename
    year = file_path[-8:-4]
    
    # load each season into a df
    data = pd.read_csv(file_path)
    
    # add year column
    data['Year'] = int(year)
    
    # add df to list
    dfs.append(data)

# stack dataframes together
blocking_df = pd.concat(dfs, axis=0, ignore_index=True)

# drop all columns except ['team_name', 'Year', 'grades_pass_block', 'grades_run_block', 'snap_counts_pass_block', 'snap_counts_run_block']
blocking_df = blocking_df.drop([col for col in blocking_df.columns if col not in ['player', 'team_name', 'Year', 'grades_pass_block', 'grades_run_block', 'snap_counts_pass_block', 'snap_counts_run_block']], axis=1)

# team name map
team_map = dict(zip(['ARZ', 'ATL', 'BLT', 'BUF', 'CAR', 'CHI', 'CIN', 'CLV', 'DAL', 'DEN', 'DET', 'GB', 'HST', 'IND', 
                     'JAX', 'KC', 'LA', 'LAC', 'LV', 'MIA', 'MIN', 'NO', 'NE', 'NYG', 'NYJ', 'OAK', 'PHI', 'PIT', 'SD', 'SEA', 'SF', 'SL', 'TB', 'TEN', 'WAS'], 
                     ['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL', 'DEN', 'DET', 'GNB', 'HOU', 'IND', 
                      'JAX', 'KAN', 'LAR', 'LAC', 'LVR', 'MIA', 'MIN', 'NOR', 'NWE', 'NYG', 'NYJ', 'OAK', 'PHI', 'PIT', 'SDG', 'SEA', 'SFO', 'STL', 'TAM', 'TEN', 'WAS']))

# map blocking_df team names
blocking_df['team_name'] = blocking_df['team_name'].map(team_map)

# reorder
blocking_df = blocking_df[['player', 'team_name', 'Year', 'grades_pass_block', 'grades_run_block', 'snap_counts_pass_block', 'snap_counts_run_block']]

# check
blocking_df.head()

,player,team_name,Year,grades_pass_block,grades_run_block,snap_counts_pass_block,snap_counts_run_block
0,Scott Wells,GNB,2006,86.9,81.0,687,427
1,Jahri Evans,NOR,2006,80.1,83.7,632,467
2,Alex Barron,STL,2006,70.8,53.2,679,419
3,Logan Mankins,NWE,2006,89.9,78.0,596,490
4,Dan Koppen,NWE,2006,84.7,63.9,594,489


Now, we will multiply each offensive lineman's grade with their number of snaps.

In [14]:
# multiply grade by snap count to get total grade
blocking_df['Tot_pass_block_grade'] = blocking_df['grades_pass_block'] * blocking_df['snap_counts_pass_block']
blocking_df['Tot_run_block_grade'] = blocking_df['grades_run_block'] * blocking_df['snap_counts_run_block']

# check
blocking_df.head()

,player,team_name,Year,grades_pass_block,grades_run_block,snap_counts_pass_block,snap_counts_run_block,Tot_pass_block_grade,Tot_run_block_grade
0,Scott Wells,GNB,2006,86.9,81.0,687,427,59700.3,34587.0
1,Jahri Evans,NOR,2006,80.1,83.7,632,467,50623.2,39087.9
2,Alex Barron,STL,2006,70.8,53.2,679,419,48073.2,22290.8
3,Logan Mankins,NWE,2006,89.9,78.0,596,490,53580.4,38220.0
4,Dan Koppen,NWE,2006,84.7,63.9,594,489,50311.8,31247.1


In [16]:
# Group by 'team_name' and 'Year' and calculate the team-level pass and run block grades
blocking_group = blocking_df.groupby(['team_name', 'Year']).apply(
    lambda x: pd.Series({
        'OL_pass_block_grade': x['Tot_pass_block_grade'].sum() / x['snap_counts_pass_block'].sum(),
        'OL_run_block_grade': x['Tot_run_block_grade'].sum() / x['snap_counts_run_block'].sum()
    })
).reset_index()

# Check the resulting grouped DataFrame
blocking_group.head()

,team_name,Year,OL_pass_block_grade,OL_run_block_grade
0,ARI,2006,69.076177,62.732493
1,ARI,2007,73.210022,64.707227
2,ARI,2008,66.187641,73.132646
3,ARI,2009,63.399149,74.350826
4,ARI,2010,66.696147,67.446229


In [18]:
# merge

 I actually end up not using these calculated grades, as the __total team__ pass & run block grades are contained in the [pff_data/team_data](./data/pff_data/team_data/2006.csv) directory. These team blocking grades include the grades of all players (OL + TE + RB + WR).

## Team Data

In [59]:
# get all team data files
file_paths = [os.path.join('./data/pff_data/team_data', file) for file in os.listdir('./data/pff_data/team_data') if file.endswith('.csv')]

# list to hold dfs
dfs = []

# read each file into a dataframe
for file_path in file_paths:
    # get year as string from filename
    year = file_path[-8:-4]
    
    # load each season into a df
    data = pd.read_csv(file_path)
    
    # add year column
    data['Year'] = int(year)
    
    # add df to list
    dfs.append(data)

# stack dataframes together
team_df = pd.concat(dfs, axis=0, ignore_index=True)

# reorder
team_df = team_df[['Tm', 'Year'] + team_df.columns[1:-1].tolist()]

# calculate PPG and PPG allowed
team_df['PPG'] = team_df['Points For'] / (team_df['Wins'] + team_df['Losses'])
team_df['PPG_allowed'] = team_df['Points Against'] / (team_df['Wins'] + team_df['Losses'])

# drop columns
team_df = team_df.drop(columns=['Points For', 'Points Against'])

# add 'Team' to the beginning of each column name
team_df.columns = ['Team_' + col for col in team_df.columns]

# remove 'Receiving' from 'Tm' and 'Year'
team_df['Tm'] = team_df['Team_Tm']
team_df['Year'] = team_df['Team_Year']
team_df = team_df.drop(columns=['Team_Tm', 'Team_Year'])

# check
team_df.head()

,Team_Team Grade,Team_Offense Grade,Team_Passing Grade,Team_Pass Blocking Grade,Team_Receiving Grade,Team_Rushing Grade,Team_Run Blocking Grade,Team_Defense Grade,Team_Run Defense Grade,Team_Tackling Grade,Team_Pass Rush Grade,Team_Coverage Grade,Team_Special Teams Grade,Team_Wins,Team_Losses,Team_PPG,Team_PPG_allowed,Tm,Year
0,67.5,65.3,54.3,72.2,78.1,39.0,63.1,67.6,66.3,64.5,58.9,76.4,NaN,5.0,11.0,19.6250,24.3125,ARI,2006
1,74.6,71.3,59.4,92.3,59.0,69.7,77.3,73.0,78.7,89.1,67.4,70.9,NaN,7.0,9.0,18.2500,20.5000,ATL,2006
2,86.1,72.5,61.4,91.6,79.7,48.6,67.2,91.4,92.7,88.8,70.9,92.3,NaN,13.0,3.0,22.0625,12.5625,BAL,2006
3,71.8,64.8,45.4,80.6,77.6,41.4,63.5,77.4,55.6,89.9,62.7,93.1,72.2,7.0,9.0,18.7500,19.4375,BUF,2006
4,81.3,72.3,55.8,85.3,77.6,44.8,77.7,86.3,76.1,76.8,74.3,92.2,NaN,8.0,8.0,16.8750,19.0625,CAR,2006


In [60]:
# merge with all 4 positional subsets
qb = qb.merge(team_df, on=['Tm', 'Year'], how='left')
rb = rb.merge(team_df, on=['Tm', 'Year'], how='left')
wr = wr.merge(team_df, on=['Tm', 'Year'], how='left')
te = te.merge(team_df, on=['Tm', 'Year'], how='left')

# check shapes
qb.shape, rb.shape, wr.shape, te.shape

((4148, 172), (9321, 173), (9841, 161), (5478, 161))

## Save

In [61]:
# save
qb.to_csv('./data/final_data/qb.csv', index=False)
rb.to_csv('./data/final_data/rb.csv', index=False)
wr.to_csv('./data/final_data/wr.csv', index=False)
te.to_csv('./data/final_data/te.csv', index=False)